1.이미지 얼굴 검출하기

In [20]:
import numpy as np
import cv2

font = cv2.FONT_HERSHEY_SIMPLEX
def faceDetect(frame):
    eye_detect = True
    face_cascade = cv2.CascadeClassifier('datasets/haarcascade_frontface.xml')
    eye_cascade = cv2.CascadeClassifier('datasets/haarcascade_eye.xml')
    info = ''
    
            
    if eye_detect:
        info = 'Eye Detection On'
    else:
        info = 'Eye Detection Off'
            
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    cv2.putText(frame, info, (5, 15), font, 0.5, (255,0,255), 1)
    
    face_number, xywh = faces.shape
    sub_img = {}

    
    for i in range(face_number):
        x, y, w, h = faces[i]
        sub_img[i] = gray[y:y+w,x:x+h] 
        cv2.imshow('face%s'%i,sub_img[i])      
        
    cv2.waitKey(0)
    
    k = cv2.waitKey(0) & 0xFF
    if k ==27:  #ESC
        cv2.destroyAllWindows()  
    elif k == ord('c'):
        for i in range(face_number):
            savedir = 'facesets/people' + str(i) + '.jpg'
            cv2.imwrite(savedir,sub_img[i])
        cv2.destroyAllWindows()

2.이미지 사이즈 바꾸기

In [2]:
def resizing(img,L=244):
    
    img2 = cv2.resize(img, (L,L), fx=1, fy=1, interpolation=cv2.INTER_AREA)
    
    return img2

3.detect face_landmark

In [3]:
#change from coordinate to np 
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68,2), dtype=dtype)
    
    #2-tuple of (x,y)-coordinates
    for i in range(68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
        
    return coords

In [4]:
def landmark_detect(frame):
    
    #face detector(HOG-based), facial landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = resizing(gray,L=244)
    rects = detector(gray, 1)
    board = np.zeros((244,244,3), np.uint8)


    for (i,rect) in enumerate(rects):
    
#landmark shape 예측
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)

        for (x, y) in shape:
            cv2.circle(board, (x, y), 1, (0, 0, 255), -1)

# show the output image with the face detections + facial landmarks
    cv2.imshow("Output", board)
    cv2.waitKey(0)
    
    k = cv2.waitKey(0) & 0xFF
    if k ==27:  #ESC
        cv2.destroyAllWindows()  
    elif k == ord('c'):
        cv2.imwrite('marksets/save',board)
        cv2.destroyAllWindows()

In [22]:
import cv2
import numpy as np
import dlib

imgfile = 'facesets/people30.jpg' #이미지 저장 디렉토리
frame = cv2.imread(imgfile, cv2.IMREAD_COLOR) # 이미지 불러오기

faceDetect(frame)
landmark_detect(frame)

AttributeError: 'tuple' object has no attribute 'shape'

4.face_emotion 분류하기(by microsoft face)

In [11]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
#%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO

# Replace <Subscription Key> with your valid subscription key.
subscription_key = "173bd422f7a94a07887cd4a29b3c29fb"
assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the westcentralus region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect"

# Set image_path to the local path of an image that you want to analyze.
image_path = "image/lena.jpg"

# Read the image into a byte array
image_data = open(image_path, "rb").read()
headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion'
}
response = requests.post(
    vision_base_url, headers=headers, params=params, data=image_data)
response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
analysis = response.json()
print(analysis)
emotion = analysis[0]['faceAttributes']['emotion']
print(emotion)

[{'faceRectangle': {'top': 887, 'left': 846, 'width': 612, 'height': 612}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.0, 'neutral': 1.0, 'sadness': 0.0, 'surprise': 0.0}}}]
{'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.0, 'neutral': 1.0, 'sadness': 0.0, 'surprise': 0.0}


In [15]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
#%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO

#  ms_faceAPI를 사용하여 emotion dict로 추출
def face_classification(img_path):
    subscription_key = "173bd422f7a94a07887cd4a29b3c29fb"
    assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the westcentralus region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
    vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect"

# Set image_path to the local path of an image that you want to analyze.
#     image_path = "image/lena.jpg"

# Read the image into a byte array
    image_data = open(image_path, "rb").read()
    headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
    params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion'
}
    response = requests.post(
    vision_base_url, headers=headers, params=params, data=image_data)
    response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
    analysis = response.json()
    print(analysis)
    emotion = analysis[0]['faceAttributes']['emotion']
    return emotion
    

5.face_emotion저장하기

In [16]:
import numpy as np
import cv2

def save_emotion(image_path,emotion):
    frame = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    #data 정제(score, class)
    emotion_list = [(k,emotion[k]) for k in emotion]
    emotion_score = [emotion[k] for k in emotion]
    emotion_class = [k for k in emotion]

    emotion_score = np.array(emotion_score)
    selected_class = emotion_class[np.argmax(emotion_score)]

    cv2.imwrite('image/emotion/'+selected_class+'/name.jpg',frame)
    